In [7]:
# # Import necessary libraries
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# from dash import Dash, html, dcc, Input, Output
# from plotly.subplots import make_subplots

# # Load the Excel data
# file_path = 'LY Case Study Quantitative_11.11.24.xlsx'
# df_week1 = pd.read_excel(file_path, sheet_name="W-1")
# df_week2 = pd.read_excel(file_path, sheet_name="W")

# # Define exchange rates
# exchange_rates = {
#     'France': 1.06, 'United Kingdom': 1.27, 'USA (tax free)': 1.0,
#     'Japan': 0.0065, 'Hong Kong': 0.13, 'China': 0.14,
#     'Korea': 0.00071, 'Korea DF': 0.00071, 'Hainan DF': 0.14,
#     'United Arab Emirates': 3.67, 'Australia': 0.65, 'Canada': 0.72,
#     'Macao': 0.12, 'Mexico': 0.049, 'Saudi Arabia': 0.27,
#     'Singapore': 0.75, 'Thailand': 0.029, 'Taiwan': 0.031
# }


# # Data cleaning and preparation function
# def clean_data(df):
#     df.dropna(how='all', inplace=True)
#     df.dropna(axis=1, how='all', inplace=True)
#     df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
#     price_columns = [col for col in df.columns if col not in ['brand', 'name', 'category', 'sku', 'url']]
#     df[price_columns] = df[price_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
#     return df, price_columns

# df_week1, price_columns_week1 = clean_data(df_week1)
# df_week2, price_columns_week2 = clean_data(df_week2)

# # Standardize columns and merge data
# df_week1 = df_week1.add_suffix('_week1')
# df_week2 = df_week2.add_suffix('_week2')
# for col in ['brand', 'name', 'category', 'sku']:
#     df_week1.rename(columns={f"{col}_week1": col}, inplace=True)
#     df_week2.rename(columns={f"{col}_week2": col}, inplace=True)
# df_merged = pd.merge(df_week1, df_week2, on=['brand', 'name', 'sku'], how='left')

# # Calculate price differences and percentage changes
# for region in price_columns_week1:
#     week1_col = f"{region}_week1"
#     week2_col = f"{region}_week2"
#     diff_col = f"{region}_price_difference"
#     pct_change_col = f"{region}_percentage_change"
#     df_merged[diff_col] = df_merged[week2_col] - df_merged[week1_col]
#     df_merged[pct_change_col] = ((df_merged[week2_col] - df_merged[week1_col]) / df_merged[week1_col]) * 100

# # Outlier detection function
# def detect_outliers(df, price_columns):
#     outliers_all_regions = pd.DataFrame()
#     for region in price_columns:
#         diff_col = f"{region}_price_difference"
#         Q1 = df[diff_col].quantile(0.25)
#         Q3 = df[diff_col].quantile(0.75)
#         IQR = Q3 - Q1
#         region_outliers = df[(df[diff_col] < (Q1 - 1.5 * IQR)) | (df[diff_col] > (Q3 + 1.5 * IQR))]
#         region_outliers = region_outliers[['brand', 'name', diff_col]].copy()
#         region_outliers['Region'] = region
#         region_outliers.rename(columns={diff_col: 'Price Difference'}, inplace=True)
#         outliers_all_regions = pd.concat([outliers_all_regions, region_outliers], ignore_index=True)
#     return outliers_all_regions

# outliers_df = detect_outliers(df_merged, price_columns_week1)


# # Top Movers Analysis
# top_movers = {}
# for region in price_columns_week1:
#     pct_change_col = f"{region}_percentage_change"
#     top_increase = df_merged.nlargest(5, pct_change_col)[['brand', 'name', pct_change_col]]
#     top_decrease = df_merged.nsmallest(5, pct_change_col)[['brand', 'name', pct_change_col]]
#     top_movers[region] = {'Top Increase': top_increase, 'Top Decrease': top_decrease}



# # Define visualization functions
# def generate_heatmap():
#     brand_analysis = df_merged.groupby('brand').agg({f"{region}_price_difference": 'mean' for region in price_columns_week1}).reset_index()
#     brand_analysis_long = brand_analysis.melt(id_vars="brand", var_name="region", value_name="Avg_Price_Diff")
#     brand_analysis_long['region'] = brand_analysis_long['region'].str.replace('_price_difference', '')
#     fig = px.imshow(
#         brand_analysis_long.pivot(index="brand", columns="region", values="Avg_Price_Diff"),
#         labels=dict(x="Region", y="Brand", color="Avg Price Difference"),
#         title="Average Price Difference by Brand and Region"
#     )
#     fig.update_xaxes(side="top")
#     return fig

# def generate_grouped_bar_chart():
#     brand_analysis = df_merged.groupby('brand').agg({f"{region}_price_difference": 'mean' for region in price_columns_week1}).reset_index()
#     brand_analysis_long = brand_analysis.melt(id_vars="brand", var_name="region", value_name="Avg_Price_Diff")
#     brand_analysis_long['region'] = brand_analysis_long['region'].str.replace('_price_difference', '')
#     fig = px.bar(
#         brand_analysis_long,
#         x="brand",
#         y="Avg_Price_Diff",
#         color="region",
#         barmode="group",
#         title="Average Price Difference by Brand across Regions",
#         labels={"Avg_Price_Diff": "Average Price Difference", "brand": "Brand", "region": "Region"}
#     )
#     fig.update_layout(xaxis_tickangle=-45)
#     return fig

# def generate_region_avg_change_bar():
#     region_variability = pd.DataFrame({
#         'Region': [region.replace('_week1', '') for region in price_columns_week1],
#         'Average Percentage Change': [df_merged[f"{region}_percentage_change"].mean() for region in price_columns_week1]
#     })
#     fig = px.bar(
#         region_variability, x='Region', y='Average Percentage Change',
#         title='Average Percentage Price Change by Region',
#         labels={'Average Percentage Change': 'Average % Price Change'}
#     )
#     return fig

# def generate_price_change_distribution():
#     price_diff_data = pd.concat([
#         df_merged[[f"{region}_price_difference"]].rename(columns={f"{region}_price_difference": "Price Difference"}).assign(Region=region)
#         for region in price_columns_week1
#     ], ignore_index=True)
#     fig = px.box(price_diff_data, x='Region', y='Price Difference', title='Distribution of Price Changes by Region')
#     return fig

# def generate_outliers_scatter():
#     fig = px.scatter(
#         outliers_df,
#         x="Region",
#         y="Price Difference",
#         color="brand",
#         hover_data=["name"],
#         title="Outliers in Price Difference by Region",
#         labels={"Price Difference": "Price Difference", "Region": "Region"}
#     )
#     fig.update_traces(marker=dict(size=8))
#     return fig

# # Visualization function for Top Movers Analysis
# def generate_top_movers(region):
#     increase_data = top_movers[region]['Top Increase']
#     decrease_data = top_movers[region]['Top Decrease']
#     fig = go.Figure()
    
#     fig.add_trace(
#         go.Bar(
#             x=increase_data['name'], 
#             y=increase_data[f"{region}_percentage_change"], 
#             name=f"{region} - Increase", 
#             marker_color='green'
#         )
#     )
    
#     fig.add_trace(
#         go.Bar(
#             x=decrease_data['name'], 
#             y=decrease_data[f"{region}_percentage_change"], 
#             name=f"{region} - Decrease", 
#             marker_color='red'
#         )
#     )
    
#     fig.update_layout(
#         title=f"Top Price Increases and Decreases for {region}",
#         xaxis_title="Product",
#         yaxis_title="Percentage Change (%)",
#         barmode='group'
#     )
    
#     return fig

# # Initialize Dash app
# app = Dash(__name__)

# # Define layout with tabs
# app.layout = html.Div([
#     html.H1("Pricing Analysis Dashboard"),
#     dcc.Tabs([
#         dcc.Tab(label='Average Price Difference Heatmap', children=[
#             dcc.Graph(figure=generate_heatmap())
#         ]),
#         dcc.Tab(label='Average Price Difference by Brand', children=[
#             dcc.Graph(figure=generate_grouped_bar_chart())
#         ]),
#         dcc.Tab(label='Average Percentage Change by Region', children=[
#             dcc.Graph(figure=generate_region_avg_change_bar())
#         ]),
#         dcc.Tab(label='Price Change Distribution by Region', children=[
#             dcc.Graph(figure=generate_price_change_distribution())
#         ]),
#         dcc.Tab(label='Outliers in Price Difference', children=[
#             dcc.Graph(figure=generate_outliers_scatter())
#         ]),
#         dcc.Tab(label='Top Movers Analysis', children=[
#             html.P("Select a region to view top increases and decreases in price percentage change:"),
#             dcc.Dropdown(
#                 id='region-dropdown',
#                 options=[{'label': region, 'value': region} for region in top_movers.keys()],
#                 value=list(top_movers.keys())[0]
#             ),
#             dcc.Graph(id='top-movers-graph')
#         ])
#     ])
# ])

# # Callback for updating Top Movers Analysis graph
# @app.callback(
#     Output('top-movers-graph', 'figure'),
#     Input('region-dropdown', 'value')
# )
# def update_top_movers_graph(selected_region):
#     return generate_top_movers(selected_region)
    

# # Run the Dash app
# if __name__ == '__main__':
#     app.run_server(debug=True)


In [1]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output

# Load the Excel data
file_path = 'LY Case Study Quantitative_11.11.24.xlsx'
df_week1 = pd.read_excel(file_path, sheet_name="W-1")
df_week2 = pd.read_excel(file_path, sheet_name="W")

# Define exchange rates for conversion to USD
exchange_rates = {
    'France': 1.06,  # EUR to USD
    'United Kingdom': 1.27,  # GBP to USD
    'USA (tax free)': 1.0,  # USD to USD
    'Japan': 0.0065,  # JPY to USD
    'Hong Kong': 0.13,  # HKD to USD
    'China': 0.14,  # CNY to USD
    'Korea': 0.00071,  # KRW to USD
    'Korea DF': 0.00071,
    'Hainan DF': 0.14,
    'United Arab Emirates': 3.67,  # AED to USD
    'Australia': 0.65,  # AUD to USD
    'Canada': 0.72,  # CAD to USD
    'Macao': 0.12,  # MOP to USD
    'Mexico': 0.049,  # MXN to USD
    'Saudi Arabia': 0.27,  # SAR to USD
    'Singapore': 0.75,  # SGD to USD
    'Thailand': 0.029,  # THB to USD
    'Taiwan': 0.031  # TWD to USD
}

# Data cleaning and preparation function
def clean_data(df):
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    price_columns = [col for col in df.columns if col not in ['brand', 'name', 'category', 'sku', 'url']]
    df[price_columns] = df[price_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
    return df, price_columns

df_week1, price_columns_week1 = clean_data(df_week1)
df_week2, price_columns_week2 = clean_data(df_week2)

# Apply exchange rates to convert prices to USD
for country in exchange_rates.keys():
    week1_col = f"{country.lower().replace(' ', '_')}_week1"
    week2_col = f"{country.lower().replace(' ', '_')}_week2"
    
    if week1_col in df_week1.columns:
        df_week1[week1_col] = df_week1[week1_col] * exchange_rates[country]
    if week2_col in df_week2.columns:
        df_week2[week2_col] = df_week2[week2_col] * exchange_rates[country]

# Merge data on SKU
df_merged = pd.merge(df_week1, df_week2, on=['sku', 'brand', 'name'], suffixes=('_week1', '_week2'))

# Calculate price differences and percentage changes in USD
for region in exchange_rates.keys():
    region_col = region.lower().replace(' ', '_')
    week1_col = f"{region_col}_week1"
    week2_col = f"{region_col}_week2"
    price_diff_col = f"{region_col}_price_difference"
    pct_change_col = f"{region_col}_percentage_change"
    
    if week1_col in df_merged.columns and week2_col in df_merged.columns:
        df_merged[price_diff_col] = df_merged[week2_col] - df_merged[week1_col]
        df_merged[pct_change_col] = (df_merged[price_diff_col] / df_merged[week1_col]) * 100

# Prepare DataFrame for average percentage change by region
avg_pct_change_usd = pd.DataFrame({
    'Region': [region for region in exchange_rates.keys()],
    'Average Percentage Change (USD)': [
        df_merged[f"{region.lower().replace(' ', '_')}_percentage_change"].mean() 
        for region in exchange_rates.keys() if f"{region.lower().replace(' ', '_')}_percentage_change" in df_merged.columns
    ]
})

# Prepare data for box plot of percentage changes by region
pct_change_data_usd = pd.concat([
    df_merged[[f"{region.lower().replace(' ', '_')}_percentage_change"]].rename(
        columns={f"{region.lower().replace(' ', '_')}_percentage_change": "Percentage Change (USD)"}).assign(Region=region)
    for region in exchange_rates.keys() if f"{region.lower().replace(' ', '_')}_percentage_change" in df_merged.columns
], ignore_index=True)

# Outlier detection function
def detect_outliers(df, price_columns):
    outliers_all_regions = pd.DataFrame()
    for region in price_columns:
        diff_col = f"{region}_price_difference"
        if diff_col in df.columns:
            Q1 = df[diff_col].quantile(0.25)
            Q3 = df[diff_col].quantile(0.75)
            IQR = Q3 - Q1
            region_outliers = df[(df[diff_col] < (Q1 - 1.5 * IQR)) | (df[diff_col] > (Q3 + 1.5 * IQR))]
            region_outliers = region_outliers[['brand', 'name', diff_col]].copy()
            region_outliers['Region'] = region
            region_outliers.rename(columns={diff_col: 'Price Difference'}, inplace=True)
            outliers_all_regions = pd.concat([outliers_all_regions, region_outliers], ignore_index=True)
    return outliers_all_regions

outliers_df = detect_outliers(df_merged, [region.lower().replace(' ', '_') for region in exchange_rates.keys()])


# Top Movers Analysis
top_movers = {}
for region in exchange_rates.keys():
    pct_change_col = f"{region.lower().replace(' ', '_')}_percentage_change"
    if pct_change_col in df_merged.columns:
        top_increase = df_merged.nlargest(5, pct_change_col)[['brand', 'name', pct_change_col]]
        top_decrease = df_merged.nsmallest(5, pct_change_col)[['brand', 'name', pct_change_col]]
        top_movers[region] = {'Top Increase': top_increase, 'Top Decrease': top_decrease}

# Generate Heatmap of Average Price Differences by Brand and Region
def generate_brand_price_difference_heatmap():
    # Group by brand and calculate mean price difference for each region
    brand_analysis = df_merged.groupby('brand').agg(
        {f"{region}_price_difference": 'mean' for region in price_columns_week1}
    ).reset_index()

    # Melt the data for heatmap format
    brand_analysis_long = brand_analysis.melt(id_vars="brand", var_name="region", value_name="Avg_Price_Diff")
    brand_analysis_long['region'] = brand_analysis_long['region'].str.replace('_price_difference', '')

    # Pivot data for heatmap
    brand_analysis_pivot = brand_analysis_long.pivot(index="brand", columns="region", values="Avg_Price_Diff")

    # # Plot heatmap
    # fig = px.imshow(
    #     brand_analysis_pivot,
    #     labels=dict(x="Region", y="Brand", color="Avg Price Difference"),
    #     title="Average Price Difference by Brand and Region"
    # )
    # Plot heatmap with the "RdBu" color scale for readability
    fig = px.imshow(
        brand_analysis_pivot,
        labels=dict(x="Region", y="Brand", color="Avg Price Difference"),
        title="Average Price Difference by Brand and Region",
        color_continuous_scale="RdBu",  # Diverging scale for clear positive/negative differentiation
        zmin=-brand_analysis_pivot.abs().max().max(),  # Ensure both sides of the scale are balanced
        zmax=brand_analysis_pivot.abs().max().max()
    )
    fig.update_xaxes(side="top")
    return fig

    
# Percentage Change Heatmap
def generate_pct_change_heatmap():
    # Identify available columns for percentage change in USD
    available_pct_change_columns = [col for col in df_merged.columns if '_percentage_change' in col]
    regions_in_data = [col.replace('_percentage_change', '') for col in available_pct_change_columns]
    
    # Calculate the average percentage change for each region
    avg_pct_change_data = df_merged[available_pct_change_columns].mean().to_frame().reset_index()
    avg_pct_change_data.columns = ['Region', 'Average Percentage Change (USD)']
    avg_pct_change_data['Region'] = regions_in_data  # Update with actual region names
    
    # Create a heatmap with each region as a column and the average percentage change as rows
    fig = px.imshow(
        avg_pct_change_data[['Average Percentage Change (USD)']].T,
        labels=dict(x="Region", y="Metric", color="Avg Percentage Change (%)"),
        x=avg_pct_change_data['Region'],
        y=['Avg % Change'],
        color_continuous_scale="RdBu",
        title="Average Percentage Price Change by Region (Standardized to USD)"
    )
    fig.update_xaxes(side="top")
    fig.update_layout(coloraxis_colorbar=dict(title="Avg % Change"))
    
    return fig


def generate_grouped_bar_chart():
    brand_analysis = df_merged.groupby('brand').agg(
        {f"{region}_price_difference": 'mean' for region in price_columns_week1}
    ).reset_index()
    brand_analysis_long = brand_analysis.melt(id_vars="brand", var_name="region", value_name="Avg_Price_Diff")
    brand_analysis_long['region'] = brand_analysis_long['region'].str.replace('_price_difference', '')
    fig = px.bar(
        brand_analysis_long,
        x="brand",
        y="Avg_Price_Diff",
        color="region",
        barmode="group",
        title="Average Price Difference by Brand across Regions",
        labels={"Avg_Price_Diff": "Average Price Difference", "brand": "Brand", "region": "Region"}
    )
    fig.update_layout(xaxis_tickangle=-45)
    return fig
    

# Visualization functions (simplified for space)
def generate_average_change_bar():
    fig = px.bar(avg_pct_change_usd, x='Region', y='Average Percentage Change (USD)',
                 title='Average Percentage Price Change by Region (in USD)')
    return fig

def generate_pct_change_box():
    fig = px.box(pct_change_data_usd, x='Region', y='Percentage Change (USD)',
                 title='Distribution of Percentage Price Changes by Region (Standardized to USD)')
    return fig

def generate_outliers_scatter():
    fig = px.scatter(outliers_df, x="Region", y="Price Difference", color="brand", hover_data=["name"],
                     title="Outliers in Price Difference by Region")
    fig.update_traces(marker=dict(size=8))
    return fig


def generate_top_movers_dropdown():
    fig = go.Figure()
    for region, movers in top_movers.items():
        increase_data = movers['Top Increase']
        decrease_data = movers['Top Decrease']
        
        fig.add_trace(
            go.Bar(
                x=increase_data['name'], 
                y=increase_data[f"{region.lower().replace(' ', '_')}_percentage_change"], 
                name=f"{region} - Increase", 
                marker_color='green',
                visible=False
            )
        )
        
        fig.add_trace(
            go.Bar(
                x=decrease_data['name'], 
                y=decrease_data[f"{region.lower().replace(' ', '_')}_percentage_change"], 
                name=f"{region} - Decrease", 
                marker_color='red',
                visible=False
            )
        )

    initial_region = list(top_movers.keys())[0]
    for trace in fig.data[:2]:
        trace.visible = True

    dropdown_buttons = [
        {
            'label': region,
            'method': 'update',
            'args': [
                {'visible': [i // 2 == j for i in range(len(top_movers) * 2)]},
                {'title': f"Top Price Increases and Decreases for {region}"}
            ]
        }
        for j, region in enumerate(top_movers)
    ]

    fig.update_layout(
        title=f"Top Price Increases and Decreases for {initial_region}",
        xaxis_title="Product",
        yaxis_title="Percentage Change (%)",
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': "down",
            'showactive': True,
        }]
    )

    return fig
    

# Dash application setup
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Pricing Analysis Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Average Price Difference Heatmap', children=[
            dcc.Graph(figure=generate_brand_price_difference_heatmap())  # Using new function for brand and region heatmap
        ]),
        dcc.Tab(label='Average Percentage Change Heatmap', children=[
            dcc.Graph(figure=generate_pct_change_heatmap())
        ]),
        dcc.Tab(label='Average Price Change by Region (USD)', children=[dcc.Graph(figure=generate_average_change_bar())]),
        dcc.Tab(label='Price Change Distribution by Region', children=[dcc.Graph(figure=generate_pct_change_box())]),
        dcc.Tab(label='Outliers in Price Difference', children=[dcc.Graph(figure=generate_outliers_scatter())]),
        dcc.Tab(label='Top Price Movers by Region', children=[dcc.Graph(figure=generate_top_movers_dropdown())])

    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)
